In [1]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2023-09-19", end="2023-09-21", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

C:\Users\nagan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-09-19 00:00:00,1.069404,1.069519,1.069404,1.069519,1.069519,0
2023-09-19 00:15:00,1.069633,1.069747,1.069519,1.069633,1.069633,0
2023-09-19 00:30:00,1.069519,1.069633,1.069519,1.069633,1.069633,0
2023-09-19 00:45:00,1.069633,1.069633,1.069519,1.069519,1.069519,0
2023-09-19 01:00:00,1.069633,1.069976,1.069633,1.069747,1.069747,0
...,...,...,...,...,...,...
2023-09-20 22:45:00,1.066667,1.066894,1.066553,1.066894,1.066894,0
2023-09-20 23:00:00,1.066667,1.066667,1.066325,1.066325,1.066325,0
2023-09-20 23:15:00,1.066212,1.066212,1.065984,1.065984,1.065984,0


In [2]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bear market
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bull market
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
 
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [3]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    179
2      7
1      6
Name: count, dtype: int64

In [4]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [5]:

def get_candles(n):
    
    access_token= '711c31656939cd2a3883c45235efcd6a-91fc6012144b44126ad8644f26f19411'
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [6]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    accountID= "101-001-26738343-001"
    client = API(access_token= '711c31656939cd2a3883c45235efcd6a-91fc6012144b44126ad8644f26f19411')
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-2000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=2000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [7]:
trading_job()

      Open    Close     High      Low
0  1.06458  1.06410  1.06461  1.06409
1  1.06410  1.06443  1.06446  1.06408
1.0652000000000001    1.06406    1.06368    1.06482
{'orderCreateTransaction': {'id': '80', 'accountID': '101-001-26738343-001', 'userID': 26738343, 'batchID': '80', 'requestID': '43140502194109639', 'time': '2023-09-24T13:37:22.900594150Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '2000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.06520', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.06406', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '81', 'accountID': '101-001-26738343-001', 'userID': 26738343, 'batchID': '80', 'requestID': '43140502194109639', 'time': '2023-09-24T13:37:22.900594150Z', 'type': 'ORDER_CANCEL', 'orderID': '80', 'reason': 'MARKET_HALTED'}, 'relatedTransactionIDs': ['80', '81'], 'lastTransactionID': '81'}
